# Slurm Environment Test Notebook

This notebook emulates the exact Slurm environment to test both LoRA and Full Fine-tuning before submitting to Rangpur.

## What This Tests:
- ✅ `torchrun` with `--standalone --nproc_per_node=1`
- ✅ CUDA multiprocessing with `spawn` method
- ✅ Model loading order (datasets first, then model)
- ✅ Both LoRA and Full FT training strategies
- ✅ Automatic evaluation after training
- ✅ All the fixes we applied

## Usage:
1. Run the setup cell
2. Choose which test to run (LoRA or Full FT)
3. Monitor for any errors
4. If successful, submit to Rangpur with confidence!


## 1. Environment Setup (Mirrors Slurm)


In [ ]:
# Import required libraries
import os
import sys
import subprocess
import multiprocessing
import torch
from pathlib import Path

# Set up environment variables (mirrors Slurm)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU
os.environ['HF_HOME'] = '/content/hf_cache'  # HuggingFace cache
os.environ['TRANSFORMERS_CACHE'] = '/content/hf_cache'

# Create cache directory
Path('/content/hf_cache').mkdir(exist_ok=True)

print("🔧 Environment Setup Complete")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"Python: {sys.executable}")
print(f"PyTorch: {torch.__version__}")
print(f"Multiprocessing start method: {multiprocessing.get_start_method()}")


## 2. Clone Repository (Mirrors Slurm Script)


In [ ]:
# Clone the repository (mirrors Slurm script behavior)
import subprocess
import os

# Set project root (mirrors Slurm)
PROJECT_ROOT = "/content/comp3710/a3"
REPO_URL = "https://github.com/nchung/comp3710-a3.git"  # Replace with your actual repo

# Remove existing directory if it exists
if os.path.exists(PROJECT_ROOT):
    subprocess.run(["rm", "-rf", PROJECT_ROOT], check=True)

# Clone repository
print(f"📥 Cloning repository to {PROJECT_ROOT}...")
subprocess.run(["git", "clone", REPO_URL, PROJECT_ROOT], check=True)

# Change to project directory
os.chdir(PROJECT_ROOT)
print(f"✅ Repository cloned and changed to: {os.getcwd()}")

# List contents to verify
print("\n📁 Repository contents:")
subprocess.run(["ls", "-la"], check=True)


## 3. Install Dependencies (Mirrors Slurm)


In [ ]:
# Install dependencies (mirrors Slurm script)
print("📦 Installing dependencies...")

# Install from requirements.txt
subprocess.run(["pip", "install", "-r", "requirements.txt"], check=True)

# Install additional packages that might be needed
subprocess.run(["pip", "install", "torch", "torchvision", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cu118"], check=True)

print("✅ Dependencies installed")


## 4. Test Configuration Files


In [ ]:
# Test that config files exist and are valid
import yaml

configs_to_test = [
    "configs/train_flant5_base_lora.yaml",
    "configs/train_t5_small_full.yaml"
]

for config_path in configs_to_test:
    print(f"\n🔍 Testing {config_path}...")
    
    if not os.path.exists(config_path):
        print(f"❌ Config file not found: {config_path}")
        continue
    
    try:
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
        
        print(f"✅ Config loaded successfully")
        print(f"   Model: {config.get('model', {}).get('name', 'Not specified')}")
        print(f"   Strategy: {config.get('training', {}).get('strategy', 'Not specified')}")
        print(f"   Output dir: {config.get('output_dir', 'Not specified')}")
        
    except Exception as e:
        print(f"❌ Error loading config: {e}")

print("\n✅ Configuration testing complete")


## 5. Test LoRA Training (Mirrors Slurm Script)


In [ ]:
# Test LoRA training with torchrun (exact Slurm command)
print("🚀 Testing LoRA Training with torchrun...")
print("This mirrors: conda run -n torch torchrun --standalone --nproc_per_node=1 src/train.py configs/train_flant5_base_lora.yaml")

# Set multiprocessing start method to spawn (our fix)
multiprocessing.set_start_method('spawn', force=True)
print(f"✅ Multiprocessing start method set to: {multiprocessing.get_start_method()}")

# Run the exact command from Slurm script
cmd = [
    "python", "-m", "torch.distributed.run",  # This is torchrun
    "--standalone",
    "--nproc_per_node=1",
    "src/train.py",
    "configs/train_flant5_base_lora.yaml"
]

print(f"\n🔧 Running command: {' '.join(cmd)}")
print("\n📊 Training output:")
print("=" * 80)

try:
    # Run the training command
    result = subprocess.run(cmd, capture_output=False, text=True, cwd=PROJECT_ROOT)
    
    if result.returncode == 0:
        print("\n" + "=" * 80)
        print("✅ LoRA Training completed successfully!")
    else:
        print("\n" + "=" * 80)
        print(f"❌ LoRA Training failed with exit code: {result.returncode}")
        
except Exception as e:
    print(f"\n❌ Error running LoRA training: {e}")

print("\n🔍 Checking for output files...")
if os.path.exists("checkpoints/flan-t5-base-lora-biolaysumm"):
    print("✅ LoRA checkpoint directory created")
    subprocess.run(["ls", "-la", "checkpoints/flan-t5-base-lora-biolaysumm"], check=True)
else:
    print("❌ LoRA checkpoint directory not found")


## 6. Test Full Fine-tuning Training (Mirrors Slurm Script)


In [ ]:
# Test Full Fine-tuning with torchrun (exact Slurm command)
print("🚀 Testing Full Fine-tuning Training with torchrun...")
print("This mirrors: conda run -n torch torchrun --standalone --nproc_per_node=1 src/train.py configs/train_t5_small_full.yaml")

# Set multiprocessing start method to spawn (our fix)
multiprocessing.set_start_method('spawn', force=True)
print(f"✅ Multiprocessing start method set to: {multiprocessing.get_start_method()}")

# Run the exact command from Slurm script
cmd = [
    "python", "-m", "torch.distributed.run",  # This is torchrun
    "--standalone",
    "--nproc_per_node=1",
    "src/train.py",
    "configs/train_t5_small_full.yaml"
]

print(f"\n🔧 Running command: {' '.join(cmd)}")
print("\n📊 Training output:")
print("=" * 80)

try:
    # Run the training command
    result = subprocess.run(cmd, capture_output=False, text=True, cwd=PROJECT_ROOT)
    
    if result.returncode == 0:
        print("\n" + "=" * 80)
        print("✅ Full Fine-tuning Training completed successfully!")
    else:
        print("\n" + "=" * 80)
        print(f"❌ Full Fine-tuning Training failed with exit code: {result.returncode}")
        
except Exception as e:
    print(f"\n❌ Error running Full Fine-tuning training: {e}")

print("\n🔍 Checking for output files...")
if os.path.exists("checkpoints/t5-small-full-biolaysumm"):
    print("✅ Full FT checkpoint directory created")
    subprocess.run(["ls", "-la", "checkpoints/t5-small-full-biolaysumm"], check=True)
else:
    print("❌ Full FT checkpoint directory not found")


## 7. Test Evaluation Script (Mirrors Slurm)


In [ ]:
# Test evaluation script (mirrors Slurm eval_runner calls)
print("🔍 Testing Evaluation Script...")

# Test LoRA evaluation
if os.path.exists("checkpoints/flan-t5-base-lora-biolaysumm"):
    print("\n📊 Testing LoRA Evaluation...")
    cmd = ["python", "src/eval_runner.py", "configs/train_flant5_base_lora.yaml"]
    
    try:
        result = subprocess.run(cmd, capture_output=False, text=True, cwd=PROJECT_ROOT)
        if result.returncode == 0:
            print("✅ LoRA Evaluation completed successfully!")
        else:
            print(f"❌ LoRA Evaluation failed with exit code: {result.returncode}")
    except Exception as e:
        print(f"❌ Error running LoRA evaluation: {e}")
else:
    print("⚠️ Skipping LoRA evaluation - no checkpoint found")

# Test Full FT evaluation
if os.path.exists("checkpoints/t5-small-full-biolaysumm"):
    print("\n📊 Testing Full FT Evaluation...")
    cmd = ["python", "src/eval_runner.py", "configs/train_t5_small_full.yaml"]
    
    try:
        result = subprocess.run(cmd, capture_output=False, text=True, cwd=PROJECT_ROOT)
        if result.returncode == 0:
            print("✅ Full FT Evaluation completed successfully!")
        else:
            print(f"❌ Full FT Evaluation failed with exit code: {result.returncode}")
    except Exception as e:
        print(f"❌ Error running Full FT evaluation: {e}")
else:
    print("⚠️ Skipping Full FT evaluation - no checkpoint found")


## 8. Results Summary


In [ ]:
# Summary of test results
print("📋 SLURM ENVIRONMENT TEST SUMMARY")
print("=" * 50)

# Check what was created
checkpoints_dir = Path("checkpoints")
if checkpoints_dir.exists():
    print("\n📁 Checkpoint directories created:")
    for item in checkpoints_dir.iterdir():
        if item.is_dir():
            print(f"   ✅ {item.name}")
            
            # Check for reports
            reports_dir = item / "reports"
            if reports_dir.exists():
                print(f"      📊 Reports: {list(reports_dir.glob('*'))}")
            else:
                print(f"      ⚠️ No reports directory")
else:
    print("❌ No checkpoints directory found")

print("\n🎯 Next Steps:")
print("1. If all tests passed, your Slurm scripts should work on Rangpur")
print("2. Submit both training jobs to Rangpur")
print("3. Monitor the logs for any issues")
print("4. Check the results in the checkpoint directories")

print("\n✅ Slurm environment test complete!")
